In [1]:
data_var = '2024-01-22'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10386,2024-01-22,Brasil Nbb,19:30,Fortaleza B.C.,São José,1.95,1.77,154.5,1.84,1.86,1.5,1.81,1.90,h0zCbg9g,0.512821,0.564972,0.543478,0.537634,0.077792,0.4,0.4,NaN,NaN,217.484,130.528356,0.600175,147.836,50.605929,0.342311,123.432,158.960,126.75,191.35,0.0,0.0,0.0,0.0,0.068430,0.007644,0.034307,-2.47,-0.494,-1.923077,0.630620,0.6,-0.030620,-1.52,-0.304,-2.532895,0.000000,0.0,0.000000
10387,2024-01-22,Brasil Nbb,19:30,Unifacisa,Mogi,1.19,4.27,162.5,1.87,1.83,-11.5,2.07,1.67,63vGcDOa,0.840336,0.234192,0.534759,0.546448,0.074528,1.0,0.4,NaN,NaN,233.876,120.856365,0.516754,269.368,109.883195,0.407930,148.042,296.972,151.94,269.01,0.0,0.0,0.0,0.0,0.797761,0.015289,0.151253,2.63,0.526,0.361217,0.000000,0.0,0.000000,2.71,0.542,6.033210,0.000000,0.0,0.000000
10388,2024-01-22,Brasil Nbb,20:00,Corinthians Paulista,Brasília,1.14,5.13,157.5,1.85,1.85,-12.5,2.00,1.72,0QwKdXw6,0.877193,0.194932,0.540541,0.540541,0.072125,1.0,0.4,NaN,NaN,161.560,50.200313,0.310722,415.830,195.863635,0.471019,183.636,524.026,127.05,340.18,0.0,0.0,0.0,0.0,0.899954,0.000000,0.106446,-0.75,-0.150,-0.933333,0.602438,0.4,-0.202438,-5.00,-1.000,-4.130000,0.000000,0.0,0.000000
10389,2024-01-22,Brasil Nbb,20:00,Pinheiros,Cerrado,1.35,3.02,154.5,1.85,1.85,-7.5,2.01,1.72,vHxOeigC,0.740741,0.331126,0.540541,0.540541,0.071867,0.6,0.6,NaN,NaN,184.236,93.008111,0.504831,217.550,53.186642,0.244480,180.866,426.056,108.36,246.40,0.0,0.0,0.0,0.0,0.540443,0.000000,0.109952,-1.43,-0.286,-1.223776,0.000000,0.0,0.000000,3.35,0.670,3.014925,0.000000,0.0,0.000000
10390,2024-01-22,Brasil Nbb,20:00,R10 Score Vasco da Gama,União Corinthians,1.22,3.98,149.5,1.80,1.86,-10.5,2.00,1.72,6w9KfB8I,0.819672,0.251256,0.555556,0.537634,0.070928,0.6,0.4,NaN,NaN,180.696,99.849306,0.552582,273.930,151.224564,0.552056,137.348,288.922,97.50,197.34,0.0,0.0,0.0,0.0,0.750621,0.023184,0.106446,1.29,0.258,0.852713,0.000000,0.0,0.000000,1.39,0.278,10.719424,0.332523,0.4,0.067477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448,2024-01-22,Lituânia Moteru Lyga Feminina,14:30,Panevezys F,Aistes LSU Kaunas F,2.29,1.58,133.5,1.85,1.85,3.5,1.85,1.85,Y96Gaqui,0.436681,0.632911,0.540541,0.540541,0.069593,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.259455,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10449,2024-01-22,República Tcheca Copa Da República Tcheca,13:45,Kolin,Brno,1.66,2.13,157.5,1.85,1.85,-3.5,1.97,1.73,tpmEkJYE,0.602410,0.469484,0.540541,0.540541,0.071893,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,229.40,0.0,0.0,0.0,0.0,0.175377,0.000000,0.091733,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10450,2024-01-22,Rússia Premier League Feminina,11:00,Nadezhda F,Dynamo Kursk F,2.76,1.41,135.5,1.85,1.85,3.5,2.10,1.62,r74wNiAe,0.362319,0.709220,0.540541,0.540541,0.071539,0.0,0.0,NaN,NaN,176.478,131.862484,0.747189,96.766,24.293421,0.251053,0.000,0.000,92.04,130.24,0.0,0.0,0.0,0.0,0.457839,0.000000,0.182479,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10451,2024-01-22,Rússia Premier League Feminina,11:00,UMMC Ekaterinburg F,MBA Moscow F,1.28,3.44,138.5,1.85,1.85,-9.5,2.04,1.66,2emtOXfk,0.781250,0.290698,0.540541,0.540541,0.0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Lakers,Portland Trail Blazers,226.5,1.82,0.8318,Over
1,21:00,Eua Nba,Detroit Pistons,Milwaukee Bucks,245.5,1.80,0.9999,Over
2,21:00,Eua Nba,Orlando Magic,Cleveland Cavaliers,214.5,1.93,0.9582,Over
3,21:00,Eua Nba,Philadelphia 76ers,San Antonio Spurs,237.5,1.89,1.0000,Over
4,21:30,Eua Nba,Toronto Raptors,Memphis Grizzlies,224.5,1.80,1.0000,Over
5,22:00,Eua Nba,Minnesota Timberwolves,Charlotte Hornets,222.5,1.88,1.0000,Over
6,22:30,Eua Nba,Dallas Mavericks,Boston Celtics,239.5,1.94,0.8934,Over
7,23:00,Eua Nba,Phoenix Suns,Chicago Bulls,227.5,1.85,1.0000,Over
